<a href="https://colab.research.google.com/github/PietrH/1-lost_parents/blob/main/get_wikidata_botanical_collector_properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive to store data to
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive


Mounted at /gdrive


In [2]:
!pip install sparqlwrapper
!pip install wikidata
# https://rdflib.github.io/sparqlwrapper/

     |████████████████████████████████| 482 kB 6.4 MB/s 
     |████████████████████████████████| 41 kB 548 kB/s 


In [90]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# botanical collector
#query = """SELECT DISTINCT ?item ?itemLabel WHERE {
#  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
#  {
#    SELECT DISTINCT ?item ?property ?propertyLabel WHERE {
#      ?item p:P106 ?statement0.
#      ?statement0 (ps:P106/(wdt:P279*)) wd:Q2083925.
#      
#    }
#    LIMIT 100000
#  }
#}"""

# naturalist
#query = """SELECT DISTINCT ?item ?itemLabel WHERE {
#  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
#  {
#    SELECT DISTINCT ?item ?property ?propertyLabel WHERE {
#      ?item p:P106 ?statement0.
#      ?statement0 (ps:P106/(wdt:P279*)) wd:Q18805.
#      
#    }
#    LIMIT 100000
#  }
#}"""

# biologist
query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
  {
    SELECT DISTINCT ?item ?property ?propertyLabel WHERE {
      ?item p:P106 ?statement0.
      ?statement0 (ps:P106/(wdt:P279*)) wd:Q864503.
      
    }
    LIMIT 500
  }
}"""


def get_results(endpoint_url, query):
    user_agent = "get-botcol-props Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

#for result in results["results"]["bindings"]:
#    print(result)

In [91]:
len(results['results']['bindings'])

500

In [92]:
import requests
import re
from collections import Counter
import itertools as it

def grouper(n,iterable):
  iterable = iter(iterable)
  return iter(lambda:list(it.islice(iterable,n)), [])

q_nums = []
for result in results['results']['bindings']:
  q_nums.append(result['itemLabel']['value'])

out = []

for q_num_group in grouper(50,q_nums):
  
  #q_num = result['itemLabel']['value']
  #reply = requests.get(f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={q_num}")
  #json.loads(reply.json())
  reply = requests.get(f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={'|'.join(q_num_group)}")
  out.append(re.findall('(?<=property\":\")P[0-9]+',reply.text))

c = Counter([item for sublist in out for item in sublist])
n_prop = len(set([item for sublist in out for item in sublist]))
#c.most_common(n_prop)
c.most_common(10)

[('P248', 5389),
 ('P813', 5123),
 ('P143', 4493),
 ('P214', 2191),
 ('P1810', 1727),
 ('P854', 1601),
 ('P106', 1557),
 ('P735', 569),
 ('P569', 531),
 ('P27', 528)]

Let's use the wbgetentities api to get property information, one property and one field at a time (so two requests per property)

In [96]:
import pandas as pd

# too slow to do a large list of properties
#df = pd.DataFrame(c.most_common(n_prop),columns=['Property','n'])
df = pd.DataFrame(c.most_common(10),columns=['Property','n'])

def get_datatype(p_number):
  reply = requests.get(f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={p_number}")
  return reply.json()['entities'][p_number]["datatype"]

def get_label(p_number,lang = "en"):
  reply = requests.get(f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={p_number}")
  return reply.json()['entities'][p_number]['labels']['en']['value']


df['Label']= df['Property'].map(get_label)
df['datatype'] = df['Property'].map(get_datatype)

df


,Property,n,Label,datatype
0,P248,5389,stated in,wikibase-item
1,P813,5123,retrieved,time
2,P143,4493,imported from Wikimedia project,wikibase-item
3,P214,2191,VIAF ID,external-id
4,P1810,1727,named as,string
5,P854,1601,reference URL,url
6,P106,1557,occupation,wikibase-item
7,P735,569,given name,wikibase-item
8,P569,531,date of birth,time
9,P27,528,country of citizenship,wikibase-item


In [139]:
def get_datatype_json(json):
  return json["datatype"]

def get_label_json(json):
  return json['labels']['en']['value']

df = pd.DataFrame(c.most_common(n_prop),columns=['Property','n'])

out = {}

for p_num_group in grouper(50,list(df.Property.values)):
  reply = requests.get(f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={'|'.join(p_num_group)}")
  out.update(reply.json()['entities'])

df['json'] = df['Property'].map(out)
df['Label'] = df['json'].map(get_label_json)
df['datatype'] = df['json'].map(get_datatype_json)

df.drop('json', axis = 1)



,Property,n,Label,datatype
0,P248,5389,stated in,wikibase-item
1,P813,5123,retrieved,time
2,P143,4493,imported from Wikimedia project,wikibase-item
3,P214,2191,VIAF ID,external-id
4,P1810,1727,named as,string
...,...,...,...,...
755,P7502,1,Golden ID,external-id
756,P3954,1,Italian Senate ID,external-id
757,P9625,1,Dizionario Biografico dell'Educazione (1800–20...,external-id
758,P4521,1,Radio Radicale person ID,external-id


In [4]:
from wikidata.client import Client
import re
client = Client()

tally = []

for result in results["results"]["bindings"]:
  q_num = result["itemLabel"]["value"]
  entity = client.get(q_num, load=False)
  #list all properties
  props = re.findall("(?<=property': ')P[0-9]+",str(entity.attributes['claims']))
  tally.append(props)

In [5]:
from collections import Counter

c = Counter([item for sublist in tally for item in sublist])

In [6]:

# get n most common properties for botanical collectors, or all of them

n_prop = len(set([item for sublist in tally for item in sublist]))
#c.most_common(n_prop)
c.most_common(35)

[('P813', 192084),
 ('P248', 181921),
 ('P143', 145591),
 ('P106', 90163),
 ('P214', 83934),
 ('P854', 77603),
 ('P1810', 52976),
 ('P31', 50143),
 ('P21', 44767),
 ('P735', 43497),
 ('P27', 32909),
 ('P569', 31751),
 ('P570', 22234),
 ('P734', 21199),
 ('P69', 21060),
 ('P108', 20492),
 ('P227', 20349),
 ('P7859', 19260),
 ('P19', 18089),
 ('P691', 17486),
 ('P1412', 16632),
 ('P213', 16107),
 ('P166', 15184),
 ('P4081', 14031),
 ('P244', 13689),
 ('P580', 13296),
 ('P585', 12378),
 ('P4656', 11100),
 ('P2006', 11054),
 ('P1207', 11030),
 ('P268', 10942),
 ('P463', 10601),
 ('P1006', 10584),
 ('P20', 10534),
 ('P2671', 9838)]

In [7]:
prop = client.get('P586')

In [8]:
prop.label

m'IPNI author ID'

In [9]:
import pandas as pd

df = pd.DataFrame(c.most_common(n_prop),columns=['Property','n'])
#df['label'] = df['Property'].map()

In [10]:
df['Label']= df['Property'].map(lambda prop: client.get(prop).label.texts['en'])

In [11]:
%load_ext google.colab.data_table

Some properties can occur more than once on an entity, especially claims regarding references; Because I think it's valuable to see what claims are added to references, I included these in the tally

In [12]:
df

,Property,n,Label
0,P813,192084,retrieved
1,P248,181921,stated in
2,P143,145591,imported from Wikimedia project
3,P106,90163,occupation
4,P214,83934,VIAF ID
...,...,...,...
1927,P8566,1,monumenta.ch ID
1928,P9062,1,BABEL author ID
1929,P9455,1,Biographia Benedictina ID
1930,P4687,1,Dictionnaire des peintres belges ID


In [13]:
%unload_ext google.colab.data_table

write output to csv and download


In [14]:
df.to_csv("/gdrive/MyDrive/biologist_properties.csv")

In [15]:
from google.colab import files

files.download("/gdrive/MyDrive/biologist_properties.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>